In [1]:
import tensorflow as tf
import os
os.makedirs("/tmp/model")
os.makedirs("/tmp/model-subset")

# 建立一些變數以及對應的名字．

v1 = tf.Variable([0.1, 0.1], name="v1")
v2 = tf.Variable([0.2, 0.2], name="v2")

# 建立所有 variables 的初始化 ops
init_op = tf.global_variables_initializer()

# 建立 saver 物件
saver = tf.train.Saver()

with tf.Session() as sess:
    
    # 執行初始化
    sess.run(init_op)
    
    #重新指定 v2 的值
    ops = tf.assign(v2, [0.3, 0.3])
    sess.run(ops)
    
    print sess.run(tf.global_variables())
    # ... 中間略去許多模型定義以及訓練，例如可以是 MNIST 的定義以及訓練
    
    save_path = saver.save(sess, "/tmp/model/model.ckpt") # 儲存模型到 /tmp/model.ckpt

[array([ 0.1,  0.1], dtype=float32), array([ 0.30000001,  0.30000001], dtype=float32)]


In [ ]:
# 使用 inspect_checkpoint.py tool 來印出 ckpt 檔
!python /usr/local/lib/python2.7/dist-packages/tensorflow/python/tools/inspect_checkpoint.py --file_name=/tmp/model/model.ckpt

In [2]:
#save specific variables

tf.reset_default_graph()
v1 = tf.Variable([0.1, 0.1], name="v1")
v2 = tf.Variable([0.4, 0.4], name="v2")
init_ops = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init_ops)
    saver = tf.train.Saver({"my_v2": v2})
    save_path = saver.save(sess, "/tmp/model-subset/model.ckpt")

In [4]:
#Model restoration
#方法一(需要事先設定好model長相)
tf.reset_default_graph()

v1 = tf.Variable(tf.constant(0.1, shape = [2]), name="v1")
v2 = tf.Variable(tf.constant(0.2, shape = [2]), name="v2")

saver = tf.train.Saver()

with tf.Session() as sess:
    saver.restore(sess, "/tmp/model/model.ckpt")
    print("Model restored.")
    print("all values %s" % sess.run(tf.global_variables()))
    print("v2 value : %s" % sess.run(v2))

INFO:tensorflow:Restoring parameters from /tmp/model/model.ckpt
Model restored.
all values [array([ 0.1,  0.1], dtype=float32), array([ 0.30000001,  0.30000001], dtype=float32)]
v2 value : [ 0.30000001  0.30000001]


In [5]:
#Model restoration
#方法二(不須事先設定model樣子)
tf.reset_default_graph()
with tf.Session() as sess:
    saver = tf.train.import_meta_graph('/tmp/model-subset/model.ckpt.meta')
    saver.restore(sess, tf.train.latest_checkpoint('/tmp/model-subset/'))
    sess.run(tf.global_variables_initializer())
    all_vars = tf.trainable_variables()
    for v in all_vars:
        print("%s with value %s" % (v.name, sess.run(v)))

INFO:tensorflow:Restoring parameters from /tmp/model-subset/model.ckpt
v1:0 with value [ 0.1  0.1]
v2:0 with value [ 0.40000001  0.40000001]
